In [3]:
import googlemaps
from pymongo import MongoClient
from datetime import datetime
import os
import requests
import json
import time
import pandas as pd
import folium
from dotenv import load_dotenv
load_dotenv()

True

In [106]:
def searchPlaces(query, location):
    API_key = os.getenv('key')
    endpoint_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    places = []
    params = {
        'query': query,
        'location': location,
        'radius': 0.25,
        'key': API_key
    }
    res = requests.get(endpoint_url, params = params)
    results =  json.loads(res.content)
    places.extend(results['results'])
    time.sleep(2)
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token'],
        res = requests.get(endpoint_url, params = params)
        results = json.loads(res.content)
        places.extend(results['results'])
        time.sleep(2)
    return places

In [44]:
def getLocation(result):
    location = []
    for i in range(len(result)):
        latitude = result[i]['geometry']['location']['lat']
        longitude = result[i]['geometry']['location']['lng']
        name = result[i]['name']
        address = result[i]['formatted_address']
        loc = {
            'name': name,
            'address': address,
            'longitude': longitude,
            'latitude': latitude
        }
        location.append(loc)
    return location  

#Starbucks

In [62]:
starbucks1 = getLocation(searchPlaces('starbucks','37.566879,-122.323895'))
starbucks2 = getLocation(searchPlaces('starbucks','47.6103008,-122.3399782'))
starbucks3 = getLocation(searchPlaces('starbucks','38.960381,-77.084811'))

In [68]:
stardf1 = pd.DataFrame(starbucks1)
stardf2 = pd.DataFrame(starbucks2)
stardf3 = pd.DataFrame(starbucks3)

In [73]:
starbucks = pd.concat([stardf1,stardf2,stardf3])
starbucks.to_csv('starbucks.csv')

In [76]:
myclient = MongoClient("mongodb://localhost/")
mydb = myclient["datlab"]
mycol = mydb["starbucks"]

In [4]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [5]:
db, star = connectCollection('datlab','starbucks')

In [7]:
queryst = list(star.find())

In [8]:
def getLocation(office):
    longitude = office['longitude']
    latitude = office['latitude']
    loc = {
        'type':'Point',
        'coordinates':[longitude, latitude]
    }
    return loc

In [11]:
for office in queryst:
    value = {"$set": {'location':getLocation(office)}}
    star.update_one(office, value)

#elementary schools

In [107]:
schools1 = getLocation(searchPlaces('elementary schools','37.566879,-122.323895'))
schools2 = getLocation(searchPlaces('elementary schools','47.6103008,-122.3399782'))
schools3 = getLocation(searchPlaces('elementary schools','38.960381,-77.084811'))

In [82]:
school1 = pd.DataFrame(schools1)
school2 = pd.DataFrame(schools2)
school3 = pd.DataFrame(schools3)

In [84]:
schools = pd.concat([school1,school2,school3])
schools.to_csv('schools.csv')

In [85]:
myclient = MongoClient("mongodb://localhost/")
mydb = myclient["datlab"]
mycol = mydb["schools"]

In [22]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [23]:
db, school = connectCollection('datlab','schools')

In [28]:
querysch = list(school.find())

In [29]:
def getLocation(office):
    longitude = office['longitude']
    latitude = office['latitude']
    loc = {
        'type':'Point',
        'coordinates':[longitude, latitude]
    }
    return loc

In [30]:
for office in querysch:
    value = {"$set": {'location':getLocation(office)}}
    school.update_one(office, value)

NIGHTCLUBS

In [93]:
clubs1 = getLocation(searchPlaces('night club San Mateo','37.566879,-122.323895'))
clubs2 = getLocation(searchPlaces('night club Seattle','47.6103008,-122.3399782'))
clubs3 = getLocation(searchPlaces('night club Washington','38.960381,-77.084811'))

In [95]:
nightclub1 = pd.DataFrame(clubs1)
nightclub2 = pd.DataFrame(clubs2)
nightclub3 = pd.DataFrame(clubs3)

In [97]:
nightclubs = pd.concat([nightclub1,nightclub2,nightclub3])
nightclubs.to_csv('nightclubs.csv')

In [98]:
myclient = MongoClient("mongodb://localhost/")
mydb = myclient["datlab"]
mycol = mydb["nightclubs"]

In [27]:
db, night = connectCollection('datlab','nightclubs')

In [31]:
queryn = list(night.find())

In [32]:
for office in queryn:
    value = {"$set": {'location':getLocation(office)}}
    night.update_one(office, value)

AIRPORTS

In [102]:
air1 = getLocation(searchPlaces('airport San Mateo','37.566879,-122.323895'))
air2 = getLocation(searchPlaces('airport Seattle','47.6103008,-122.3399782'))
air3 = getLocation(searchPlaces('airport Washington','38.960381,-77.084811'))

In [104]:
airp1 = pd.DataFrame(air1)
airp2 = pd.DataFrame(air2)
airp3 = pd.DataFrame(air3)

In [109]:
airports = pd.concat([airp1,airp2,airp3])
airports.to_csv('airports.csv')

In [110]:
myclient = MongoClient("mongodb://localhost/")
mydb = myclient["datlab"]
mycol = mydb["airports"]

In [33]:
db, air = connectCollection('datlab','airports')

In [34]:
querya = list(air.find())

In [35]:
for office in querya:
    value = {"$set": {'location':getLocation(office)}}
    air.update_one(office, value)

VEGAN RESTAURANTS

In [111]:
veg1 = getLocation(searchPlaces('vegan restaurant San Mateo','37.566879,-122.323895'))
veg2 = getLocation(searchPlaces('vegan restaurant Seattle','47.6103008,-122.3399782'))
veg3 = getLocation(searchPlaces('vegan restaurant Washington','38.960381,-77.084811'))

In [115]:
vegan1 = pd.DataFrame(veg1)
vegan2 = pd.DataFrame(veg2)
vegan3 = pd.DataFrame(veg3)

In [116]:
vegans = pd.concat([vegan1,vegan2,vegan3])
vegans.to_csv('vegan_restaurants.csv')

In [117]:
myclient = MongoClient("mongodb://localhost/")
mydb = myclient["datlab"]
mycol = mydb["vegan_restaurants"]

In [36]:
db, veg = connectCollection('datlab','vegan_restaurants')

In [37]:
queryv = list(veg.find())

In [38]:
for office in queryv:
    value = {"$set": {'location':getLocation(office)}}
    veg.update_one(office, value)